In [1]:
%matplotlib inline
# !pip install stargazer
# !pip install scipy
# !pip install plotly_express
# !pip install pandas-profiling
# !pip install linearmodels
# !pip install sklearn

import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
# import sklearn

from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


In [2]:
# path="/Users/mengxi/Documents/GitHub/research_module/data/"
# data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")

path="/Users/effie/Desktop/term3/research_module/data/"
data=pd.read_stata(path+"county_clean/119_total_and_by_industry_data_county.dta")

data=data.drop(columns=['index'])

data=data.rename(columns={"ln_gdp": "gdp_growth_100"})

data




,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,toas_ratio,emp_agr_weight,emp_man_weight,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight
0,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,...,1.0,0.002310,0.210602,0.787089,0.000741,0.321257,0.678002,0.002739,0.219986,0.777276
1,2005,"Böblingen, Landkreis",DE112,08115,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,...,1.0,0.004826,0.412613,0.582561,0.001957,0.435541,0.562501,0.006725,0.435416,0.557859
2,2005,"Esslingen, Landkreis",DE113,08116,BW,504.429993,0.212459,15237.577,30208.0,61109.0,...,1.0,0.006164,0.369497,0.624339,0.002130,0.393602,0.604268,0.007894,0.389075,0.603031
3,2005,"Göppingen, Landkreis",DE114,08117,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,...,1.0,0.012873,0.394026,0.593100,0.005486,0.417749,0.576766,0.019193,0.417092,0.563714
4,2005,"Ludwigsburg, Landkreis",DE115,08118,BW,505.869995,2.257281,16209.134,32042.0,68900.0,...,1.0,0.014138,0.338531,0.647332,0.004291,0.380648,0.615061,0.018165,0.357240,0.624595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,2012,Saale-Holzland-Kreis,DEG0J,16074,TH,84.480003,2.386800,1734.716,20534.0,48577.0,...,0.0,0.042256,0.362381,0.595363,0.028101,0.363217,0.608683,NaN,NaN,NaN
2404,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.0,0.037358,0.373862,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN
2405,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,1.0,0.028015,0.338902,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN
2406,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.0,0.002846,0.296681,0.700473,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["No_ratio"].describe()

count    2408.000000
mean        0.286297
std         0.338936
min         0.000000
25%         0.000000
50%         0.166667
75%         0.500000
max         1.000000
Name: No_ratio, dtype: float64

In [4]:
data["toas_ratio"].describe()

count    2408.000000
mean        0.627292
std         0.475227
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.146924
Name: toas_ratio, dtype: float64

In [5]:
query_data = data.query('No_ratio>0.286')
data.loc[:,'county_affected_no'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('toas_ratio>0.627')
data.loc[:,'county_affected_toas'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('year>2008')
data.loc[:,'year_post'] = data.index.map(lambda x : 1 if x in query_data.index else 0)
data

,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight,county_affected_no,county_affected_toas,year_post
0,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,...,0.787089,0.000741,0.321257,0.678002,0.002739,0.219986,0.777276,1,1,0
1,2005,"Böblingen, Landkreis",DE112,08115,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,...,0.582561,0.001957,0.435541,0.562501,0.006725,0.435416,0.557859,0,1,0
2,2005,"Esslingen, Landkreis",DE113,08116,BW,504.429993,0.212459,15237.577,30208.0,61109.0,...,0.624339,0.002130,0.393602,0.604268,0.007894,0.389075,0.603031,0,1,0
3,2005,"Göppingen, Landkreis",DE114,08117,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,...,0.593100,0.005486,0.417749,0.576766,0.019193,0.417092,0.563714,1,1,0
4,2005,"Ludwigsburg, Landkreis",DE115,08118,BW,505.869995,2.257281,16209.134,32042.0,68900.0,...,0.647332,0.004291,0.380648,0.615061,0.018165,0.357240,0.624595,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,2012,Saale-Holzland-Kreis,DEG0J,16074,TH,84.480003,2.386800,1734.716,20534.0,48577.0,...,0.595363,0.028101,0.363217,0.608683,NaN,NaN,NaN,0,0,1
2404,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN,0,0,1
2405,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN,1,1,1
2406,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.700473,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1


In [6]:
data.columns

Index(['year', 'Gebietseinheit', 'EU_Code', 'Regional_key', 'Land', 'resident',
       'gdp_growth_100', 'gdp_level', 'gdp_per_inhabitant', 'gdp_per_employee',
       'gdp_per_hour_per_employed', 'ln_gva', 'gva_level', 'gva_agr',
       'gva_man', 'gva_ser', 'gva_ser_fin', 'gva_ser_trade', 'gva_ser_public',
       'employee', 'emp_agr', 'emp_man', 'emp_ser', 'emp_ser_fin',
       'emp_ser_trade', 'emp_ser_public', 'volume_of_work', 'vow_agr',
       'vow_man', 'vow_ser', 'vow_ser_fin', 'vow_ser_trade', 'vow_ser_public',
       'fixed_agr', 'fixed_man', 'fixed_ser', 'fixed_ser_trade',
       'fixed_ser_fin', 'fixed_ser_public', 'fixed_total', 'firms_no',
       'firms_aff_no', 'No_ratio', 'toas_sum_county', 'toas_aff_sum_county',
       'toas_ratio', 'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight',
       'gva_agr_weight', 'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight',
       'vow_man_weight', 'vow_ser_weight', 'county_affected_no',
       'county_affected_toas', 'year_post'

In [7]:


data["Land"].value_counts()

BY    560
NW    392
BW    296
NI    208
HE    184
RP    160
TH    104
ST     96
SN     96
BB     88
SH     88
MV     64
SL     40
HB     16
HH      8
BE      8
Name: Land, dtype: int64

In [8]:
data["affected_after_no"] = data.county_affected_no*data.year_post
data["affected_after_toas"] = data.county_affected_toas*data.year_post
data["ln_res"]=np.log(data["resident"]*1000)

data["ln_gdp_level"] = np.log(data.gdp_level*1000000)
data["ln_gdp_per_inhabitant"] = np.log(data.gdp_per_inhabitant)
data["ln_gdp_per_employee"] = np.log(data.gdp_per_employee)
data["ln_gdp_per_hour_per_employed"] = np.log(data.gdp_per_hour_per_employed)

data["ln_gva_level"] = np.log(data.gva_level*1000000)
data["ln_gva_agr"] = np.log(data.gva_agr*1000000)
data["ln_gva_man"] = np.log(data.gva_man*1000000)
data["ln_gva_ser"] = np.log(data.gva_ser*1000000)
data["ln_gva_ser_fin"] = np.log(data.gva_ser_fin*1000000)
data["ln_gva_ser_trade"] = np.log(data.gva_ser_trade*1000000)
data["ln_gva_ser_public"] = np.log(data.gva_ser_public*1000000)

data["ln_emp"]=np.log(data["employee"]*1000)

data["ln_emp_ser_fin"] = np.log(data.emp_ser_fin*1000)
data["ln_emp_ser_trade"] = np.log(data.emp_ser_trade*1000)
data["ln_emp_ser_public"] = np.log(data.emp_ser_public*1000)
data["ln_emp_agr"]=np.log(data["emp_agr"]*1000)
data["ln_emp_man"]=np.log(data["emp_man"]*1000)
data["ln_emp_ser"]=np.log(data["emp_ser"]*1000)

data["ln_vow"]=np.log(data["volume_of_work"]*1000000)

data["ln_vow_ser_fin"] = np.log(data.vow_ser_fin*1000000)
data["ln_vow_ser_trade"] = np.log(data.vow_ser_trade*1000000)
data["ln_vow_ser_public"] = np.log(data.vow_ser_public*1000000)
data["ln_vow_agr"]=np.log(data["vow_agr"]*1000000)
data["ln_vow_man"]=np.log(data["vow_man"]*1000000)
data["ln_vow_ser"]=np.log(data["vow_ser"]*1000000)

data["ln_fixed_total"] = np.log(data.fixed_total*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_trade"] = np.log(data.fixed_ser_trade*1000000)
data["ln_fixed_ser_public"] = np.log(data.fixed_ser_public*1000000)
data["ln_fixed_agr"]=np.log(data["fixed_agr"]*1000000)
data["ln_fixed_man"]=np.log(data["fixed_man"]*1000000)
data["ln_fixed_ser"]=np.log(data["fixed_ser"]*1000000)

data["gdp_growth"]=data["gdp_growth_100"]/100

df_fe = data.set_index(['Regional_key','year'])
df_fe

,,Gebietseinheit,EU_Code,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,gdp_per_hour_per_employed,ln_gva,...,ln_vow_man,ln_vow_ser,ln_fixed_total,ln_fixed_ser_fin,ln_fixed_ser_trade,ln_fixed_ser_public,ln_fixed_agr,ln_fixed_man,ln_fixed_ser,gdp_growth
Regional_key,year,,,,,,,,,,,,,,,,,,,,,
08111,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,57.240002,-2.754354,...,18.841323,20.103557,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.025955
08115,2005,"Böblingen, Landkreis",DE112,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,42.860001,-10.404957,...,18.698373,18.946178,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.102586
08116,2005,"Esslingen, Landkreis",DE113,BW,504.429993,0.212459,15237.577,30208.0,61109.0,42.930000,0.049075,...,18.743547,19.181744,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,0.002125
08117,2005,"Göppingen, Landkreis",DE114,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,38.509998,-1.470492,...,18.002480,18.303720,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.013096
08118,2005,"Ludwigsburg, Landkreis",DE115,BW,505.869995,2.257281,16209.134,32042.0,68900.0,48.599998,2.090561,...,18.595810,19.154504,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,0.022573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16074,2012,Saale-Holzland-Kreis,DEG0J,TH,84.480003,2.386800,1734.716,20534.0,48577.0,NaN,2.438582,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.023868
16076,2012,"Greiz, Landkreis",DEG0L,TH,103.880997,2.040082,1854.354,17851.0,46776.0,NaN,2.091659,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.020401
16077,2012,"Altenburger Land, Landkreis",DEG0M,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,NaN,-1.566608,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,-0.016161


In [9]:
data_check = df_fe[['emp_agr_weight','emp_man_weight','emp_ser_weight','gdp_level','gdp_per_inhabitant','gdp_per_hour_per_employed']]
data_check

,,emp_agr_weight,emp_man_weight,emp_ser_weight,gdp_level,gdp_per_inhabitant,gdp_per_hour_per_employed
Regional_key,year,,,,,,
08111,2005,0.002310,0.210602,0.787089,39624.353,68957.0,57.240002
08115,2005,0.004826,0.412613,0.582561,12993.051,35478.0,42.860001
08116,2005,0.006164,0.369497,0.624339,15237.577,30208.0,42.930000
08117,2005,0.012873,0.394026,0.593100,6077.935,23882.0,38.509998
08118,2005,0.014138,0.338531,0.647332,16209.134,32042.0,48.599998
...,...,...,...,...,...,...,...
16074,2012,0.042256,0.362381,0.595363,1734.716,20534.0,NaN
16076,2012,0.037358,0.373862,0.588780,1854.354,17851.0,NaN
16077,2012,0.028015,0.338902,0.633083,1689.681,17732.0,NaN


<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gdp


##  gdp - no



In [10]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))



                                               Model Comparison                                              
                                     (1)              (2)                       (3)                       (4)
-------------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                       PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                    2376             2376                      2376                      2376
Cov. Est.                      Clustered        Clustered                 Clustered                 Clustered
R-squared                         0.8502           0.4607                    0.3678                    0.4570
R-Squared (Within)               -0.1174           0.3148                    0.0575                    0.3155
R-Squared 

In [11]:
# gdp growth

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                  
                                   (9)           (10)
-----------------------------------------------------
Dep. Variable               gdp_growth     gdp_growth
Estimator                     PanelOLS       PanelOLS
No. Observations                  2376           2376
Cov. Est.                    Clustered      Clustered
R-squared                       0.0440         0.1374
R-Squared (Within)              0.0415         0.1975
R-Squared (Between)             0.0883        -79.223
R-Squared (Overall)             0.0440        -4.1406
F-statistic                     15.584         65.851
P-value (F-stat)                0.0000         0.0000
=====================     ============   ============
const                         -0.0510*      -1.6469**
                              (0.0307)       (0.7839)
affected_after_no               0.0065        -0.0002
                              (0.0040)       (0.0030)
year_post                   

##  gdp - toas

In [12]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                                               Model Comparison                                               
                                      (1)              (2)                       (3)                       (4)
--------------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                        PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                     2376             2376                      2376                      2376
Cov. Est.                       Clustered        Clustered                 Clustered                 Clustered
R-squared                          0.8513           0.4602                    0.3745                    0.4578
R-Squared (Within)                -0.0948           0.3077                    0.0784                    0.2842
R

In [13]:
# gdp growth

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                   
                                    (9)           (10)
------------------------------------------------------
Dep. Variable                gdp_growth     gdp_growth
Estimator                      PanelOLS       PanelOLS
No. Observations                   2376           2376
Cov. Est.                     Clustered      Clustered
R-squared                        0.0430         0.1383
R-Squared (Within)               0.0406         0.1998
R-Squared (Between)              0.0836        -91.466
R-Squared (Overall)              0.0430        -4.8071
F-statistic                      15.182         66.359
P-value (F-stat)                 0.0000         0.0000
======================     ============   ============
const                          -0.0606*      -1.8300**
                               (0.0368)       (0.7837)
affected_after_toas             -0.0002        -0.0045
                               (0.0038)       (0.0029)
year_post 

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gva

##  gva - no



In [14]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2,"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                                    Model Comparison                                   
                                     (1)              (2)            (3)            (4)
---------------------------------------------------------------------------------------
Dep. Variable               ln_gva_level     ln_gva_level     ln_gva_agr     ln_gva_agr
Estimator                       PanelOLS         PanelOLS       PanelOLS       PanelOLS
No. Observations                    2376             2376           2376           2376
Cov. Est.                      Clustered        Clustered      Clustered      Clustered
R-squared                         0.8503           0.4607         0.6690         0.2568
R-Squared (Within)               -0.1551           0.3304         0.3161         0.4279
R-Squared (Between)               0.8583           0.5043         0.6769         0.3488
R-Squared (Overall)               0.8503           0.5029         0.6690         0.3505
F-statistic                     

In [15]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


# gva - no -significant gva level agr  man fin  trade 

                                            Model Comparison                                           
                                       (9)               (10)                 (11)                 (12)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      2376               2376                 2376                 2376
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8141             0.0291               0.7789               0.1269
R-Squared (Within)                  0.1252            -0.0030              -0.2345              -0.1045
R-Squared (Between)                 0.8195             0.3878   

##  gva - toas

In [16]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2,"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))





                                    Model Comparison                                    
                                      (1)              (2)            (3)            (4)
----------------------------------------------------------------------------------------
Dep. Variable                ln_gva_level     ln_gva_level     ln_gva_agr     ln_gva_agr
Estimator                        PanelOLS         PanelOLS       PanelOLS       PanelOLS
No. Observations                     2376             2376           2376           2376
Cov. Est.                       Clustered        Clustered      Clustered      Clustered
R-squared                          0.8514           0.4602         0.6684         0.2561
R-Squared (Within)                -0.1318           0.3233         0.3136         0.4310
R-Squared (Between)                0.8593           0.5064         0.6764         0.3508
R-Squared (Overall)                0.8514           0.5050         0.6684         0.3526
F-statistic          

In [17]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


#  gva - toas  -significant  man ser  fin  trade  pubilc 

                                            Model Comparison                                            
                                        (9)               (10)                 (11)                 (12)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       2392               2392                 2392                 2392
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8475             0.0313               0.8204               0.1367
R-Squared (Within)                   0.3407             0.0384               0.1104               0.0381
R-Squared (Between)                  0.8515            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# employee

## employee no

In [18]:
df_fe[['employee', 'emp_agr', 'emp_man','emp_ser', 'emp_ser_fin', 'emp_ser_trade', 'emp_ser_public',
       'ln_emp', 'ln_emp_ser_fin','ln_emp_agr', 'ln_emp_man', 'ln_emp_ser',
       'ln_emp_ser_trade', 'ln_emp_ser_public']]

,,employee,emp_agr,emp_man,emp_ser,emp_ser_fin,emp_ser_trade,emp_ser_public,ln_emp,ln_emp_ser_fin,ln_emp_agr,ln_emp_man,ln_emp_ser,ln_emp_ser_trade,ln_emp_ser_public
Regional_key,year,,,,,,,,,,,,,,
08111,2005,472.821014,1.092,99.577,372.152,127.607,106.265,138.280,13.066472,11.756711,6.995766,11.508686,12.827058,11.573691,11.837036
08115,2005,212.382996,1.025,87.632,123.726,32.644,53.108,37.974,12.266147,10.393416,6.932448,11.380902,11.725825,10.880083,10.544657
08116,2005,249.350006,1.537,92.134,155.679,36.087,67.210,52.382,12.426613,10.493688,7.337588,11.430999,11.955551,11.115577,10.866318
08117,2005,111.858002,1.440,44.075,66.343,12.919,25.556,27.868,11.624986,9.466454,7.272398,10.693648,11.102594,10.148627,10.235234
08118,2005,235.255005,3.326,79.641,152.288,40.715,58.201,53.372,12.368425,10.614352,8.109526,11.285284,11.933529,10.971658,10.885042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16074,2012,35.710999,1.509,12.941,21.261,3.955,8.601,8.705,10.483214,8.282736,7.319202,9.468156,9.964630,9.059634,9.071653
16076,2012,39.643002,1.481,14.821,23.341,3.864,9.603,9.874,10.587669,8.259458,7.300473,9.603800,10.057967,9.169831,9.197660
16077,2012,37.409000,1.048,12.678,23.683,3.859,9.211,10.613,10.529667,8.258163,6.954639,9.447623,10.072513,9.128154,9.269835


In [19]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_agr_weight","emp_man_weight", ]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  2392           2392           2392           2392
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.9021         0.2965         0.7708         0.1827
R-Squared (Within)              0.3682        -0.0225        -0.3173         0.2587
R-Squared (Between)             0.9029         0.7768         0.7754         0.3808
R-Squared (Overall)             0.9021         0.7755         0.7708         0.3803
F-statistic                     3661.3         219.36         1604.5        

In [20]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  2392           2392           2392           2392
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8807         0.6588         0.8984         0.4761
R-Squared (Within)              0.4539         0.3942         0.4616         0.2317
R-Squared (Between)             0.8819         0.6071         0.8992         0.7795
R-Squared (Overall)             0.8807         0.6065         0.8984         0.7784
F-statistic                     3521.5         1340.9         4219.2        

In [21]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

# empl no significant: man and ser (fin trade public)


                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      2392               2392                 2392                 2392
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8221             0.1136               0.8858               0.1288
R-Squared (Within)                  0.3202             0.1077              -0.0112               0.1049
R-Squared (Between)                 0.8256             0.5438   

## employee toas

In [22]:
df_fe.columns

Index(['Gebietseinheit', 'EU_Code', 'Land', 'resident', 'gdp_growth_100',
       'gdp_level', 'gdp_per_inhabitant', 'gdp_per_employee',
       'gdp_per_hour_per_employed', 'ln_gva', 'gva_level', 'gva_agr',
       'gva_man', 'gva_ser', 'gva_ser_fin', 'gva_ser_trade', 'gva_ser_public',
       'employee', 'emp_agr', 'emp_man', 'emp_ser', 'emp_ser_fin',
       'emp_ser_trade', 'emp_ser_public', 'volume_of_work', 'vow_agr',
       'vow_man', 'vow_ser', 'vow_ser_fin', 'vow_ser_trade', 'vow_ser_public',
       'fixed_agr', 'fixed_man', 'fixed_ser', 'fixed_ser_trade',
       'fixed_ser_fin', 'fixed_ser_public', 'fixed_total', 'firms_no',
       'firms_aff_no', 'No_ratio', 'toas_sum_county', 'toas_aff_sum_county',
       'toas_ratio', 'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight',
       'gva_agr_weight', 'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight',
       'vow_man_weight', 'vow_ser_weight', 'county_affected_no',
       'county_affected_toas', 'year_post', 'affected_after_no',
 

In [23]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public
Regional_key,year,,,,,,,
08111,2005,692.291016,1.896,152.294,538.101,187.366,159.814,190.921
08115,2005,303.177002,2.039,132.008,169.130,44.600,76.967,47.563
08116,2005,354.964996,2.802,138.108,214.055,50.270,97.081,66.703
08117,2005,157.813995,3.029,65.823,88.962,18.372,35.394,35.195
08118,2005,333.500000,6.058,119.140,208.303,56.605,84.140,67.558
...,...,...,...,...,...,...,...,...
16074,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))





                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  2392           2392           2392           2392
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.9055         0.3007         0.7713         0.1838
R-Squared (Within)              0.3740        -0.1019        -0.3553         0.2765
R-Squared (Between)             0.9064         0.7883         0.7761         0.3920
R-Squared (Overall)             0.9055         0.7869         0.7713         0.3915
F-statistic                     3264.8         179.00         1609.4        

In [25]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   2392           2392           2392           2392
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8875         0.6550         0.9045         0.4719
R-Squared (Within)               0.4463         0.3571         0.4632         0.1837
R-Squared (Between)              0.8888         0.6173         0.9054         0.7849
R-Squared (Overall)              0.8875         0.6166         0.9045         0.7837
F-statistic                      3764.4         1318.2         45

In [26]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


# empl toas significant total empl agr  man ser (fin public )




                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       2392               2392                 2392                 2392
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8283             0.1187               0.8963               0.1267
R-Squared (Within)                   0.3289             0.0414              -0.0140               0.1092
R-Squared (Between)                  0.8317            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# volume_of_work

## volume_of_work no of firms


In [27]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',
       'ln_vow', 
       'ln_vow_ser_fin', 'ln_vow_ser_trade','ln_vow_ser_public', 'ln_vow_agr', 'ln_vow_man', 'ln_vow_ser'
      ]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public,ln_vow,ln_vow_ser_fin,ln_vow_ser_trade,ln_vow_ser_public,ln_vow_agr,ln_vow_man,ln_vow_ser
Regional_key,year,,,,,,,,,,,,,,
08111,2005,692.291016,1.896,152.294,538.101,187.366,159.814,190.921,20.355517,19.048574,18.889521,19.067370,14.455257,18.841323,20.103557
08115,2005,303.177002,2.039,132.008,169.130,44.600,76.967,47.563,19.529827,17.613244,18.158887,17.677566,14.527970,18.698373,18.946178
08116,2005,354.964996,2.802,138.108,214.055,50.270,97.081,66.703,19.687530,17.732919,18.391056,18.015760,14.845844,18.743547,19.181744
08117,2005,157.813995,3.029,65.823,88.962,18.372,35.394,35.195,18.876928,16.726338,17.382053,17.376415,14.923743,18.002480,18.303720
08118,2005,333.500000,6.058,119.140,208.303,56.605,84.140,67.558,19.625153,17.851608,18.247993,18.028497,15.616890,18.595810,19.154504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16074,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# data[]"vow_agr_weight","vow_man_weight","vow_ser_weight"

In [29]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_agr_weight","vow_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_agr_weight","vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  2304           2304           2304           2304
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8926         0.2931         0.7491         0.1572
R-Squared (Within)              0.1246         0.0580        -0.3463         0.2653
R-Squared (Between)             0.8937         0.8120         0.7544         0.3892
R-Squared (Overall)             0.8926         0.8110         0.7491         0.3886
F-statistic                     3183.1         207.87         1372.5        

In [30]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  2304           2304           2304           2304
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8812         0.6398         0.9019         0.5042
R-Squared (Within)              0.4510         0.5358         0.2999         0.3333
R-Squared (Between)             0.8828         0.6840         0.9030         0.8210
R-Squared (Overall)             0.8812         0.6835         0.9019         0.8201
F-statistic                     3410.7         1187.9         4226.1        

In [31]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_ser_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





#  vow no significant man ser (trade public )



                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      2304               2304                 2304                 2304
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8295             0.1424               0.8889               0.1693
R-Squared (Within)                  0.2181             0.1395              -0.0488               0.1200
R-Squared (Between)                 0.8336             0.6563   

## volume_of_work toas

In [32]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_agr_weight","vow_man_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight","vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                    ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   2304           2304           2304           2304
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8964         0.2959         0.7488         0.1642
R-Squared (Within)               0.1428         0.0172        -0.3810         0.3197
R-Squared (Between)              0.8974         0.8212         0.7542         0.3908
R-Squared (Overall)              0.8964         0.8200         0.7488         0.3904
F-statistic                      3311.5         210.63         13

In [33]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))





                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   2304           2304           2304           2304
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8867         0.6346         0.9068         0.4989
R-Squared (Within)               0.4425         0.5176         0.3028         0.3057
R-Squared (Between)              0.8883         0.6951         0.9079         0.8259
R-Squared (Overall)              0.8867         0.6945         0.9068         0.8249
F-statistic                      3596.8         1161.5         44

In [34]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_ser_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





# vow toas signficant total vow agr ser (fin trade public )



                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       2304               2304                 2304                 2304
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8346             0.1403               0.8985               0.1672
R-Squared (Within)                   0.2206             0.1155              -0.0383               0.1294
R-Squared (Between)                  0.8387            

In [35]:
# vow and empl by toas better 
# gva soso
# gdp not ok 

In [36]:
print("done")

done
